# Installing pyspark requirements

In [1]:
import os
os.environ["JAVA_HOME"] = "/lib/jvm/java-11-openjdk-amd64"
# Because otherwise custom modules import errors
import sys
sys.path.append('../')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

import plotly.graph_objs as go
import plotly.express as px

from scipy import stats
from scipy import spatial

In [3]:
import pyspark

In [4]:
from src.ranking_helpers import *
from src.make_and_plot import*
from src.pages_groups_extraction import*
from src.data_aggregation import*

## Initialize context 

In [5]:
conf = pyspark.SparkConf().setMaster("local[3]").setAll([
                                   ('spark.driver.memory','100G'),
                                   ('spark.executor.memory', '100G'),
                                   ('spark.driver.maxResultSize', '0'),
                                    ('spark.executor.cores', '3'),
                                    ('spark.local.dir', '/scratch/descourt/spark')
                                ])
# create the session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
# create the context
sc = spark.sparkContext
sc.setLogLevel('ERROR')

23/07/16 15:57:17 WARN Utils: Your hostname, iccluster060 resolves to a loopback address: 127.0.1.1; using 10.90.39.11 instead (on interface ens786f0)
23/07/16 15:57:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/16 15:57:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/16 15:57:18 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/07/16 15:57:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/16 15:57:19 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# Data

Detailed step by step data processing for Jan and Febr 2022.

In [6]:
dfs = spark.read.parquet("/scratch/descourt/processed_data/en/pageviews_en_2015-2023.parquet").withColumn('project', lit('en'))

In [7]:
df_high_volume = extract_volume(dfs.where(dfs.date.isin(['2022-01', '2022-02', '2022-03'])), high=True).select('date', 'page_id', 'page', lit('core').alias('volume')).cache()
df_low_volume = extract_volume(dfs.where(dfs.date.isin(['2022-01', '2022-02', '2022-03'])), high=False).select('date', 'page_id', 'page', lit('tail').alias('volume')).cache()
df_all = df_high_volume.union(df_low_volume).cache()

In [8]:
pd_compls = pd.DataFrame({'date' : ['2022-01', '2022-01', '2022-01', '2022-02', '2022-02', '2022-02'], 'page_id': ['-1', '-2', '-3']*2, 'page': ['other-search', 'other-internal', 'other-external']*2})
pd_compls['volume'] = pd_compls['page']
df_compl = spark.createDataFrame(pd_compls)
df_all = df_all.union(df_compl).cache()

/home/descourt/.conda/envs/wiki_sem/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/descourt/.conda/envs/wiki_sem/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [9]:
df_1 = spark.read.csv("/scratch/descourt/clickstream/en/clickstream-enwiki-2022-01.tsv.gz", sep='\t').withColumn('date', lit('2022-01'))
df_2 = spark.read.csv("/scratch/descourt/clickstream/en/clickstream-enwiki-2022-02.tsv.gz", sep='\t').withColumn('date', lit('2022-02'))
df_clickstream = df_1.union(df_2).cache()

In [10]:
df_clickstream = df_clickstream.selectExpr("_c0 as prev", "_c1 as curr", "_c2 as type", "_c3 as count", "date as date").cache()

In [11]:
# filter
df_clickstream = df_clickstream.where((df_clickstream.type != 'other') & ~df_clickstream.prev.isin(['other-other', 'other-empty'])).cache()

In [12]:
df_clickstream = df_clickstream.groupBy('prev', 'curr', 'date').agg(sum('count').alias('count')).cache()

In [13]:
df_clickstream.count()

54407316

In [14]:
# Match on volume
df_clickstream = df_clickstream.join(df_all.select('date', col('page').alias('prev'), col('volume').alias('volume_prev')), on=['date', 'prev'])\
                               .join(df_all.select('date', col('page').alias('curr'), col('volume').alias('volume_curr')), on=['date', 'curr']).cache()

In [15]:
df_clickstream.count()

53446132

In [24]:
from operator import add
from functools import reduce
df_final = df_clickstream.groupBy('date', 'curr', 'volume_curr')\
              .pivot('volume_prev').sum('count').fillna(0)\
              .withColumn('total-external',reduce(add, [col('other-external'), col('other-search') ])).withColumn('total',reduce(add, [col('other-external'), col('other-search'), col('core'), col('tail') ]))\
              .drop('other-internal')

In [26]:
df_final.show()

+-------+--------------------+-----------+-------+--------------+------------+-----+--------------+-------+
|   date|                curr|volume_curr|   core|other-external|other-search| tail|total-external|  total|
+-------+--------------------+-----------+-------+--------------+------------+-----+--------------+-------+
|2022-04|Cairo_pentagonal_...|       tail|   45.0|         114.0|       329.0| 33.0|         443.0|  521.0|
|2022-01|2021–22_New_York_...|       core| 2402.0|           0.0|       531.0|169.0|         531.0| 3102.0|
|2022-01|Jony_(footballer,...|       tail|  631.0|           0.0|       138.0| 33.0|         138.0|  802.0|
|2022-01|      Masami_Teraoka|       tail|  157.0|           0.0|       172.0|  0.0|         172.0|  329.0|
|2022-01|Norway–Russia_border|       core|  771.0|         208.0|      3279.0|259.0|        3487.0| 4517.0|
|2022-04|     Wright_v_Tatham|       tail|    0.0|           0.0|        55.0|  0.0|          55.0|   55.0|
|2022-03|   Iowa_County,_Iow

# Analysis

In [25]:
df_clickstream = spark.read.parquet("/scratch/descourt/processed_data/clickstream/en/clickstream_grouped.parquet")

In [27]:
df_analysis = df_clickstream.select('date', 'volume_curr', (col('core') / col('total') * 100).alias('frac_core'), (col('total-external') / col('total') * 100).alias('frac_ext'), (col('tail') / col('total') * 100).alias('frac_tail')).cache()

In [34]:
df_ana = df_analysis.groupBy('date', 'volume_curr').agg(avg('frac_core').alias('avg_frac_core'), (stddev('frac_core') * 1.96 / sqrt(count('*'))).alias('std_frac_core'), avg('frac_tail').alias('avg_frac_tail'), (stddev('frac_tail') * 1.96 / sqrt(count('*'))).alias('std_frac_tail'),\
                                               avg('frac_ext').alias('avg_frac_ext'), (stddev('frac_ext') * 1.96 / sqrt(count('*'))).alias('std_frac_ext'), count('*').alias('nb_dpts') ).cache()

In [36]:
df_ana_2 = df_analysis.groupBy('date', 'volume_curr').agg(percentile_approx('frac_core', 0.5).alias('med_frac_core'), percentile_approx('frac_tail', 0.5).alias('med_frac_tail'),\
                                               percentile_approx('frac_ext', 0.5).alias('med_frac_ext')).cache()

In [37]:
df_ana_2.show()

+-------+-----------+------------------+------------------+------------------+
|   date|volume_curr|     med_frac_core|     med_frac_tail|      med_frac_ext|
+-------+-----------+------------------+------------------+------------------+
|2022-03|       core|  39.0092879256966| 1.245753114382786|  57.7762139338494|
|2022-02|       tail|6.0606060606060606|               0.0|  72.8395061728395|
|2022-04|       core| 39.89983305509182|1.2396694214876034| 56.88601850235643|
|2022-01|       tail| 7.772020725388601|               0.0| 70.43478260869566|
|2022-02|       core| 38.87140287769784| 1.214416296683207|57.953608143722214|
|2022-01|       core| 40.15233949945593| 1.282051282051282| 56.55172413793104|
|2022-03|       tail| 6.896551724137931|               0.0| 71.69811320754717|
|2022-04|       tail| 8.022922636103152|               0.0| 70.83333333333334|
+-------+-----------+------------------+------------------+------------------+



In [35]:
df_ana.sort(asc('volume_curr')).show()

+-------+-----------+------------------+--------------------+------------------+--------------------+------------------+--------------------+-------+
|   date|volume_curr|     avg_frac_core|       std_frac_core|     avg_frac_tail|       std_frac_tail|      avg_frac_ext|        std_frac_ext|nb_dpts|
+-------+-----------+------------------+--------------------+------------------+--------------------+------------------+--------------------+-------+
|2022-01|       core| 41.61761943218141| 0.05837251968716376|2.8654087021925854|0.010582082469554985| 55.51697186562567|  0.0594437468620673| 709714|
|2022-02|       core| 40.81034031291655|   0.058787232240137|2.7520040040069462|0.010425767255820069|56.437655683076535| 0.05984700643604943| 708903|
|2022-03|       core| 40.97542211600518|  0.0590914329032076| 2.779690477378634| 0.01036559741301672| 56.24488740661647| 0.06013049879174849| 711522|
|2022-04|       core| 41.58756903937836| 0.05830979883556831| 2.779762387160711|0.010284279068324062